In [1]:
from datasets import load_dataset

In [ ]:

# Load The Stack dataset
dataset = load_dataset('bigcode/the-stack', data_dir='data/python', split='train')

# Filter for Python files and extract well-commented code
def is_well_commented(example):
    code = example['content']
    comments = sum(1 for line in code.splitlines() if line.strip().startswith("#"))
    return comments / max(1, len(code.splitlines())) > 0.1  # e.g., 10% of lines are comments

python_code = dataset.filter(is_well_commented)
python_code = python_code.shuffle(seed=42).select(range(0, 1_000_000)) 


In [ ]:
from googletrans import Translator

translator = Translator()

def translate_comments(code):
    lines = code.splitlines()
    translated_lines = []
    for line in lines:
        if line.strip().startswith("#"):
            comment = line.strip("#").strip()
            translated_comment = translator.translate(comment, src='en', dest='ca').text
            translated_lines.append(f"# {translated_comment}")
        else:
            translated_lines.append(line)
    return "\n".join(translated_lines)

python_code = python_code.map(lambda x: {"content": translate_comments(x['content'])})


In [29]:
import re

def extract_exercise_statements(text):
    lines = text.splitlines()

    exercises = []
    current_exercise = ""
    first = True

    exercise_start_pattern = re.compile(r'^\s*[-*]*\s*[-*]*(EXERCICI|EXERCISE)\s*[:-]*[-*]*\s*', re.IGNORECASE)
    for line in lines:
        if exercise_start_pattern.match(line):
            print("match")
            if current_exercise:
                if first:
                    current_exercise = ""
                else:
                    exercises.append(current_exercise.strip())
                    current_exercise = ""
                first = False
        else:
            current_exercise += " " + line.strip()
    
    if current_exercise:
        exercises.append(current_exercise.strip())

    return exercises

# Example usage
response_text = """
- **EXERCICI:** Crea una funció que rebi un array d'enters i retorni un nou array amb els elements ordenats de forma ascendent, però sense utilitzar la funció `sorted()`. 
- **EXERCICI:** Escriu un programa que rebi un array d'enters i calculi la mitjana aritmètica dels elements parells i la mitjana aritmètica dels elements imparells.
- **EXERCICI:** Donat un array d'enters, crea un nou array que contingui els elements que es repeteixen un nombre parell de vegades.  - **EXERCICI:** Implementa una funció que rebi un array d'enters i retorni el valor que apareix més vegades en l'array.  - **EXERCICI:** Crea un programa que rebi un array d'enters i un enter `n`. El programa ha de retornar un nou array que contingui els `n` primers elements de l'array original.  - **EXERCICI:** Escriu una funció que rebi dos arrays d'enters i retorni un nou array que contingui la intersecció dels dos arrays.  - **EXERCICI:** Implementa una funció que rebi un array de cadenes de text i retorni un nou array amb les cadenes ordenades alfabèticament en ordre invers.  - **EXERCICI:** Crea una funció que rebi un array d'enters i retorni un nou array que contingui la suma de cada parell d'elements consecutius de l'array original.  - **EXERCICI:** Donat un array d'enters, crea un nou array que contingui els elements que són primers.  - **EXERCICI:** Implementa una funció que rebi un array d'enters i retorni el valor màxim i el valor mínim de l'array sense utilitzar les funcions `max()` i `min()`.  - **EXERCICI:** Crea un programa que rebi un array de nombres reals i un nombre real `x`. El programa ha de retornar un nou array que contingui els elements de l'array original que són més grans que `x`.  - **EXERCICI:** Escriu una funció que rebi un array de cadenes de text i retorni un nou array amb les cadenes que contenen la lletra "a".  - **EXERCICI:** Implementa una funció que rebi un array d'enters i retorni un nou array que contingui els elements que són múltiples de 3.  - **EXERCICI:** Crea una funció que rebi un array d'enters i retorni la suma dels quadrats dels elements de l'array.  - **EXERCICI:** Donat un array de nombres reals, crea un nou array que contingui la mitjana mòbil de cada 3 elements consecutius de l'array original.  - **EXERCICI:** Implementa una funció que rebi un array de cadenes de text i retorni un nou array amb les cadenes ordenades per la seva longitud en ordre creixent.  - **EXERCICI:** Crea un programa que rebi un array d'enters i un enter `n`. El programa ha de retornar un nou array que contingui els elements de l'array original que són divisibles per `n`.  - **EXERCICI:** Escriu una funció que rebi un array de nombres reals i retorni un nou array amb els elements ordenats en ordre ascendent, però sense utilitzar la funció `sorted()`.  - **EXERCICI:** Implementa una funció que rebi un array d'enters i retorni un nou array que contingui els elements que són parells i que es troben en posicions imparells de l'array original.  - **EXERCICI:** Crea una funció que rebi un array d'enters i retorni un nou array que contingui els elements que són únics, és a dir, que no es repeteixen.
"""

exercise_statements = extract_exercise_statements(response_text)
for idx, exercise in enumerate(exercise_statements):
    print(f"Exercise {idx+1}: {exercise}")
